In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, PayloadSchemaType
import pandas as pd

collection_name = "Amazon-items-collection-08-hybrid"

df_items = pd.read_json("../data/meta_Electronics_1000.jsonl", lines=True)
df_items.head(3)


def preprocess_data(row):
    return f"{row['title']} {' '.join(row['features'])}"

def extract_first_large_image(row):
    return row["images"][0].get("large", '')

df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)
df_items["first_large_image"] = df_items.apply(extract_first_large_image, axis=1)
df_sample = df_items.sample(n=50, random_state=25)


data_to_embed = df_sample[["preprocessed_data", "first_large_image", "rating_number", "price", "average_rating", 'parent_asin']].to_dict(orient="records")

df_reviews = pd.read_json("../data/Electronics_1000.jsonl", lines=True)

parent_asins = set([item['parent_asin'] for item in data_to_embed])

df_reviews_filtered = df_reviews[
    df_reviews['asin'].isin(parent_asins) | df_reviews['parent_asin'].isin(parent_asins)
]

df_reviews_filtered.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
40,5,Good antenna replacement,"Great product, easy 2min install. 3 months fin...",[],B079KXLF57,B07HGQT28F,AFWRGOGF4AI2IHRX7KZ2IYL63RXA,2019-07-27 04:29:39.520,0,True
66,4,Good features and powered by mains.,"While solar-powered cameras are convenient, I'...",[],B0B4SB2CRD,B0B4SB2CRD,AFXAMSEN6MT4WQHSXCV4JRGXWODA,2022-12-26 17:57:17.749,1,False
185,5,Great to be completely wireless.,These earbuds are terrific for not having any ...,[],B0BRVCVNTP,B0C6KGG6N7,AG6SKI6EDOIS55QBTX6RHMBG75RA,2023-03-14 03:23:12.794,0,True


In [15]:
import sqlite3
from pandas import Timestamp

conn = sqlite3.connect('../data/reviews_filtered.db')
cursor = conn.cursor()

cursor.execute('''
        DROP TABLE IF EXISTS reviews
''')

# Create table for reviews if it doesn't exist
cursor.execute('''
        CREATE TABLE [reviews] (
            [rating] FLOAT,
            [title] TEXT,
            [text] TEXT,
            [images] TEXT,
            [asin] TEXT,
            [parent_asin] TEXT,
            [user_id] TEXT,
            [helpful_vote] INTEGER,
            [verified_purchase] INTEGER
        )
''')


# Use the actual columns from the CREATE TABLE statement above
expected_columns = [
    'rating', 'title', 'text', 'images', 'asin', 'parent_asin',
    'user_id', 'helpful_vote', 'verified_purchase'
]
for col in expected_columns:
    if col not in df_reviews_filtered.columns:
        df_reviews_filtered[col] = None

import json

# Insert data into the table
for _, row in df_reviews_filtered.iterrows():
    values = []
    for col in [
        'rating', 'title', 'text', 'images', 'asin', 'parent_asin',
        'user_id',  'helpful_vote', 'verified_purchase'
    ]:
        val = row[col]
        # If not a scalar (str, int, float, bool, None), dump as JSON
        if not isinstance(val, (str, int, float, bool, type(None))):
            val = json.dumps(val)
        values.append(val)
    cursor.execute('''
        INSERT OR REPLACE INTO reviews (
            rating, title, text, images, asin, parent_asin,
            user_id, helpful_vote, verified_purchase
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(values))

conn.commit()
conn.close()